In [1]:
import pandas as pd
import seaborn as se
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

In [ ]:
# Load the Excel file into a DataFrame, reading the "Data" sheet, skipping the first row, and parsing dates
Epid_DF = pd.read_excel("HIV_Epidemiology_Children_Adolescents_2022.xlsx", 
                        sheet_name="Data", 
                        skiprows=1, 
                        parse_dates=True)

# Print the shape (number of rows and columns) of the DataFrame to understand its initial size
print(Epid_DF.shape)

# Remove duplicate rows to ensure data integrity
Epid_DF.drop_duplicates(inplace=True)

# Select only relevant columns for further analysis
Epid_DF = Epid_DF[["Type", "Country/Region", "UNICEF Region", "Indicator", "Year", "Sex", "Age", "Value"]]

# Print the shape again to see the DataFrame size after column selection and duplicate removal
print(Epid_DF.shape)

# Check and print the number of missing values in each column before handling missing data
print("Number of Nulls before delete NA", Epid_DF.isna().sum())

# Drop rows where the 'Value' column has missing data, as these are likely not usable
Epid_DF.dropna(subset=['Value'], inplace=True)

# Print the number of missing values in each column after removing rows with missing 'Value'
print("---------\n", "Number of Nulls after delete NA", Epid_DF.isna().sum())

# Reset the DataFrame's index after dropping rows, to ensure it is sequential
Epid_DF = Epid_DF.reset_index(drop=True)

# Print the final shape of the DataFrame to confirm the number of rows and columns after all cleaning steps
print(Epid_DF.shape)


In [ ]:
# Remove commas from the 'Value' column to ensure it can be processed as a numeric type
Epid_DF['Value'] = Epid_DF['Value'].str.replace(',', '')

# Convert 'Value' column to string to handle cases where it starts with '<'
Epid_DF['Value'] = Epid_DF['Value'].astype(str)

# Loop through each row in the DataFrame to handle values starting with '<'
for x in range(Epid_DF.shape[0]):
    # Check if the first character of 'Value' is '<'
    if Epid_DF.loc[x, 'Value'][0] == '<':  
        # Replace the '<' value with half of the number that follows it
        # Convert the numeric part after '<' to float, divide by 2, and store it as a string
        Epid_DF.loc[x, 'Value'] = str(float(Epid_DF.loc[x, 'Value'][1:]) / 2)

# Convert the 'Value' column back to float for numeric analysis
Epid_DF['Value'] = Epid_DF['Value'].astype(float)

# Change several columns to string type for consistency and future compatibility
# Python may deprecate the 'category' type in certain contexts, making string type safer for general use
Epid_DF['Country/Region'] = Epid_DF['Country/Region'].astype('string')
Epid_DF['UNICEF Region'] = Epid_DF['UNICEF Region'].astype('string')
Epid_DF['Indicator'] = Epid_DF['Indicator'].astype('string')
Epid_DF['Sex'] = Epid_DF['Sex'].astype('string')
Epid_DF['Age'] = Epid_DF['Age'].astype('string')

# Convert the 'Year' column to datetime format for easier time-based operations
Epid_DF['Year1'] = pd.to_datetime(Epid_DF['Year'], format="%Y")

# Print the data types of all columns to confirm the changes
print(Epid_DF.dtypes)

In [ ]:
#All rows with blank Unicef_Region, Have Type "Region". And All region type, have blank Unicef_Region. So, I want to fill all 
#Blank Unicef Region with a word "Global"

Epid_DF["UNICEF Region"].fillna("Global", inplace = True)

#All rows with "NA" Unicef_Region, Have country "United states". And All rows with country "United states", 
#have "NA" Unicef_Region. So, I want to fill all NA Unicef Region with a word "United States"

Epid_DF.loc[Epid_DF["Country/Region"]=="United States" , 'UNICEF Region']="United States"

In [ ]:
# Filter the dataset to create a subset (Epid_DF_A1) based on specific conditions:
Epid_DF_A1 = Epid_DF[
    (Epid_DF["Indicator"] == "Estimated number of annual AIDS-related deaths") &  # Include only rows where the indicator is "Estimated number of annual AIDS-related deaths"
    (Epid_DF["UNICEF Region"] != "Global") &                                      # Exclude rows where the UNICEF Region is "Global"
    (Epid_DF["Sex"] == "Both") &                                                  # Include only rows where the sex category is "Both" (aggregated data for both genders)
    ((Epid_DF["Age"] == "Age 0-14") | (Epid_DF["Age"] == "Age 15-19"))            # Include rows where age is either "Age 0-14" or "Age 15-19"
]

In [ ]:
# Filter the dataset to create a subset (Epid_DF_A2) based on specific conditions:
Epid_DF_A2 = Epid_DF[
    (Epid_DF["Indicator"] == "Estimated number of annual new HIV infections") &  # Include only rows where the indicator is "Estimated number of annual new HIV infections"
    (Epid_DF["UNICEF Region"] != "Global") &                                     # Exclude rows where the UNICEF Region is "Global"
    (Epid_DF["Sex"] == "Both") &                                                 # Include only rows where the sex category is "Both" (aggregated data for both genders)
    ((Epid_DF["Age"] == "Age 0-14") | (Epid_DF["Age"] == "Age 15-19"))           # Include rows where age is either "Age 0-14" or "Age 15-19"
]

In [ ]:
# Filter the dataset to create a subset (Epid_DF_A3) based on specific conditions:
Epid_DF_A3 = Epid_DF[
    (Epid_DF["Indicator"] == "Estimated number of people living with HIV") &   # Include only rows where the indicator is "Estimated number of people living with HIV"
    (Epid_DF["UNICEF Region"] != "Global") &                                  # Exclude rows where the UNICEF Region is "Global"
    (Epid_DF["Sex"] == "Both") &                                              # Include only rows where the sex category is "Both" (aggregated data for both genders)
    ((Epid_DF["Age"] == "Age 0-14") | (Epid_DF["Age"] == "Age 15-19"))        # Include rows where age is either "Age 0-14" or "Age 15-19"
]

In [ ]:
# Create a crosstab to calculate the mean of 'Value' for each combination of 'Year1' and 'UNICEF Region'
# The crosstab provides a pivoted view of the data, where:
# - Rows represent years ('Year1')
# - Columns represent UNICEF regions
# - Cell values represent the mean of the 'Value' column (annual AIDS-related deaths) for each year-region combination
mycrosstab = pd.crosstab(
    Epid_DF_A1["Year1"], 
    Epid_DF_A1["UNICEF Region"], 
    values=Epid_DF_A1["Value"], 
    aggfunc='mean'
)

# Import Plotly Express for creating interactive visualizations
import plotly.express as px

# Assign the crosstab DataFrame to 'df' for plotting
df = mycrosstab

# Create an area chart using Plotly Express
# - The 'facet_col' argument creates a separate subplot for each UNICEF region
# - 'facet_col_wrap=2' arranges the subplots in 2 columns for better readability
fig = px.area(
    df, 
    facet_col="UNICEF Region", 
    facet_col_wrap=2
)

# Update the chart layout to add a descriptive title
fig.update_layout(
    title_text="Estimated number of annual AIDS-related deaths in each region"
)

# Display the interactive area chart
fig.show()


In [ ]:
# Create a crosstab to calculate the mean of 'Value' for each combination of 'Year' and 'UNICEF Region'
# - Rows represent 'Year'
# - Columns represent 'UNICEF Region'
# - Cell values contain the mean of the 'Value' column (annual new HIV infections) for each year-region combination
mycrosstab = pd.crosstab(
    Epid_DF_A2["Year"], 
    Epid_DF_A2["UNICEF Region"], 
    values=Epid_DF_A2["Value"], 
    aggfunc='mean'
)

# Assign the crosstab DataFrame to 'df' for easier reference in the plot
df = mycrosstab

# Create an area chart using Plotly Express to visualize trends over time
# - 'facet_col' creates a subplot for each UNICEF region
# - 'facet_col_wrap=2' arranges subplots in 2 columns for better readability
fig = px.area(
    df, 
    facet_col="UNICEF Region", 
    facet_col_wrap=2
)

# Update the chart layout to include a title that explains the data being visualized
fig.update_layout(
    title_text="Estimated number of annual new HIV infections in each Region"
)

# Display the interactive area chart
fig.show()


In [ ]:
# Create a crosstab to calculate the mean of 'Value' for each combination of 'Year1' and 'UNICEF Region'
# - Rows represent 'Year1' (datetime version of the year)
# - Columns represent 'UNICEF Region'
# - Cell values contain the mean of the 'Value' column (number of people living with HIV) for each year-region combination
mycrosstab = pd.crosstab(
    Epid_DF_A3["Year1"], 
    Epid_DF_A3["UNICEF Region"], 
    values=Epid_DF_A3["Value"], 
    aggfunc='mean'
)

# Import Plotly Express for visualization
import plotly.express as px

# Assign the crosstab DataFrame to 'df' for use in plotting
df = mycrosstab

# Create an area chart using Plotly Express to visualize trends in people living with HIV over time
# - 'facet_col' creates a subplot for each UNICEF region
# - 'facet_col_wrap=2' arranges the subplots in 2 columns for better readability
fig = px.area(
    df, 
    facet_col="UNICEF Region", 
    facet_col_wrap=2
)

# Update the chart layout to include a descriptive title
fig.update_layout(
    title_text="Estimated number of people living with HIV in each Region"
)

# Display the interactive area chart
fig.show()


In [ ]:
# Filter the dataset to create a subset (Epid_DF_A1_Glob) for global data on annual AIDS-related deaths:
Epid_DF_A1_Glob = Epid_DF[
    (Epid_DF["Indicator"] == "Estimated number of annual AIDS-related deaths") &  # Include only rows with the indicator "Estimated number of annual AIDS-related deaths"
    (Epid_DF["UNICEF Region"] == "Global") &                                      # Focus on rows where the UNICEF Region is "Global"
    (Epid_DF["Sex"] == "Both") &                                                  # Include rows where the sex category is "Both" (aggregated for both genders)
    ((Epid_DF["Age"] == "Age 0-14") | (Epid_DF["Age"] == "Age 15-19"))            # Include rows where age is either "Age 0-14" or "Age 15-19"
]

# Group the global dataset by 'Year' and calculate the mean 'Value' for each year
x = Epid_DF_A1_Glob.groupby(Epid_DF_A1_Glob["Year"])["Value"].mean()

# Plot the mean annual global AIDS-related deaths as a bar chart
x.plot(
    kind="bar",                        # Create a bar chart
    color=(0.2, 0.4, 0.6, 0.6)         # Set bar color using RGBA values (semi-transparent blue)
)

# Set the x-axis label
plt.xlabel("Year")

# Set the y-axis label
plt.ylabel("Global Value")

# Set the title of the plot
plt.title("Estimated number of annual AIDS-related deaths globally")

# Display the plot
plt.show()

In [ ]:
# Filter the dataset to create a subset (Epid_DF_A1_Glob) for global data on people living with HIV:
Epid_DF_A1_Glob = Epid_DF[
    (Epid_DF["Indicator"] == "Estimated number of people living with HIV") &  # Include only rows with the indicator "Estimated number of people living with HIV"
    (Epid_DF["UNICEF Region"] == "Global") &                                 # Focus on rows where the UNICEF Region is "Global"
    (Epid_DF["Sex"] == "Both") &                                             # Include rows where the sex category is "Both" (aggregated for both genders)
    ((Epid_DF["Age"] == "Age 0-14") | (Epid_DF["Age"] == "Age 15-19"))       # Include rows where age is either "Age 0-14" or "Age 15-19"
]

# Group the global dataset by 'Year' and calculate the mean 'Value' for each year
x = Epid_DF_A1_Glob.groupby(Epid_DF_A1_Glob["Year"])["Value"].mean()

# Plot the mean number of people living with HIV globally as a bar chart
x.plot(
    kind="bar",                        # Create a bar chart
    color=(0.2, 0.4, 0.6, 0.6)         # Set bar color using RGBA values (semi-transparent blue)
)

# Set the x-axis label to "Year"
plt.xlabel("Year")

# Set the y-axis label to "Global Value" to indicate what the values represent
plt.ylabel("Global Value")

# Set the title of the plot to provide context on the data being visualized
plt.title("Estimated number of people living with HIV globally")

# Display the plot
plt.show()

In [ ]:
# Filter the dataset to create a subset (Epid_DF_A1_Glob2) for global data on annual AIDS-related deaths for non-both sexes:
Epid_DF_A1_Glob2 = Epid_DF[
    (Epid_DF["Indicator"] == "Estimated number of annual AIDS-related deaths") &  # Include only rows with the indicator "Estimated number of annual AIDS-related deaths"
    (Epid_DF["UNICEF Region"] == "Global") &                                   # Focus on rows where the UNICEF Region is "Global"
    (Epid_DF["Sex"] != "Both") &                                               # Exclude rows where the sex category is "Both" (separate data for males and females)
    ((Epid_DF["Age"] == "Age 0-14") | (Epid_DF["Age"] == "Age 15-19"))         # Include rows where age is either "Age 0-14" or "Age 15-19"
]

# Group the data by 'Sex' and calculate the total sum of 'Value' for each sex (male and female)
Elem = Epid_DF_A1_Glob2.groupby(['Sex'])["Value"].sum()

# Define labels for the pie chart (Female, Male)
labels = ['Female', 'Male']

# Define colors for the pie chart (a soft pink for females, and a soft blue for males)
colors = [(1, 0.1, 0.6, 0.6), (0.1, 0.5, 0.9, 0.6)]

# Create a pie chart to show the distribution of annual AIDS-related deaths between females and males
plt.pie(
    Elem,                     # Data values (total deaths for each sex)
    labels=labels,             # Labels for the pie chart (Female, Male)
    autopct='%1.1f%%',         # Display percentage with one decimal place
    colors=colors              # Set the colors for the chart (soft pink for female, soft blue for male)
)

# Set the title of the pie chart to explain the data being visualized
plt.title("Estimated number of annual AIDS-related deaths 2000-2021")

# Display the pie chart
plt.show()

In [ ]:
# Filter the dataset to create a subset (Epid_DF_A1_Glob) for global data on annual AIDS-related deaths
# - Focus on data from 2015 onward for children and adolescents (ages 0-14 and 15-19)
# - Exclude data where the sex is "Both" (separate male and female data)
# - Include only data from the global region
Epid_DF_A1_Glob = Epid_DF[
    (Epid_DF["Indicator"] == "Estimated number of annual AIDS-related deaths") &  # Filter for AIDS-related deaths
    (Epid_DF["UNICEF Region"] == "Global") &                                   # Filter for global region
    (Epid_DF["Sex"] != "Both") &                                               # Exclude "Both" sex category (focus on male and female)
    ((Epid_DF["Age"] == "Age 0-14") | (Epid_DF["Age"] == "Age 15-19")) &        # Filter for age group 0-14 and 15-19
    (Epid_DF["Year"] >= 2015)                                                  # Focus on data from 2015 onward
]

# Import Plotly Express for visualization
import plotly.express as px

# Filter the dataset for male data only
df1 = Epid_DF_A1_Glob[Epid_DF_A1_Glob['Sex'] == "Male"]

# Group the male data by 'Year' and sum the 'Value' (annual AIDS-related deaths)
tempmale = pd.DataFrame(df1.groupby(['Year']).agg({'Value': 'sum'}))

# Filter the dataset for female data only
df2 = Epid_DF_A1_Glob[Epid_DF_A1_Glob['Sex'] == "Female"]

# Group the female data by 'Year' and sum the 'Value' (annual AIDS-related deaths)
tempfemale = pd.DataFrame(df2.groupby(['Year']).agg({'Value': 'sum'}))

# Concatenate the male and female data into a single DataFrame
final = pd.concat([tempmale, tempfemale], axis=1)

# Rename the columns to 'Male' and 'Female' for clarity
final.columns = ['Male', 'Female']

# Calculate the total sum of AIDS-related deaths (male + female)
sumcolumns = final['Male'] + final['Female']

# Create a funnel chart using Plotly to visualize the total annual AIDS-related deaths by sex from 2015-2021
fig = px.funnel(final, title="Estimated Annual AIDS-Related Deaths by Sex 2015-2021")

# Display the funnel chart
fig.show()

In [ ]:
# Importing the necessary library for plotting
import matplotlib.pyplot as plt

# Filter the dataset to create a subset (Epid_DF_A3_Glob) for global data on AIDS-related deaths and people living with HIV
Epid_DF_A3_Glob = Epid_DF[
    ((Epid_DF["Indicator"] == "Estimated number of annual AIDS-related deaths") |  # Include rows where the indicator is either AIDS-related deaths or people living with HIV
    (Epid_DF["Indicator"] == "Estimated number of people living with HIV")) & 
    (Epid_DF["UNICEF Region"] == "Global") &   # Focus on the global region
    (Epid_DF["Sex"] == "Both") &               # Include both sexes (aggregate male and female data)
    ((Epid_DF["Age"] == "Age 0-14") | (Epid_DF["Age"] == "Age 15-19"))   # Focus on age groups 0-14 and 15-19 years
]

# Create a cross-tabulation (mycrosstab) to compare the mean 'Value' (number of people affected) for each year, broken down by indicator (AIDS-related deaths vs people living with HIV)
mycrosstab = pd.crosstab(
    Epid_DF_A3_Glob["Year"],             # Group by 'Year'
    Epid_DF_A3_Glob["Indicator"],       # Group by 'Indicator' (AIDS-related deaths or people living with HIV)
    values=Epid_DF_A3_Glob["Value"],    # Use the 'Value' column for aggregation
    aggfunc='mean'                      # Aggregate by taking the mean value for each combination of Year and Indicator
)

# Plot the cross-tabulated data as a bar chart
mycrosstab.plot(kind="bar")

# Set labels and title for the plot
plt.xlabel("Year")                                # Set x-axis label to "Year"
plt.ylabel("Value")                               # Set y-axis label to "Value"
plt.title("People that have HIV vs People died cause of HIV")  # Set the plot title to reflect the data being compared

# Display the plot
plt.show()

In [ ]:
# Check the relationship between Key Performance Indicators (KPIs) in each UNICEF Region
# Filter the dataset to focus on global data (UNICEF Region = "Global"), for both sexes ("Both") 
# and for the age groups 0-14 and 15-19 years
Epid_DF_A3_Glob = Epid_DF[
    (Epid_DF["UNICEF Region"] == "Global") &                  # Focus on the global region
    (Epid_DF["Sex"] == "Both") &                              # Include both sexes (aggregate male and female)
    ((Epid_DF["Age"] == "Age 0-14") | (Epid_DF["Age"] == "Age 15-19"))  # Focus on age groups 0-14 and 15-19 years
]

# Create a cross-tabulation (mycrosstab) to compare the sum of 'Value' (number of people affected) 
# for each year, broken down by indicator (e.g., incidence rate, annual deaths, etc.)
mycrosstab = pd.crosstab(
    Epid_DF_A3_Glob["Year"],              # Group by 'Year'
    Epid_DF_A3_Glob["Indicator"],        # Group by 'Indicator' (e.g., annual deaths, annual new infections, etc.)
    values=Epid_DF_A3_Glob["Value"],     # Use the 'Value' column for aggregation
    aggfunc='sum'                         # Aggregate by summing the values for each combination of Year and Indicator
)

# Rename the columns for clarity to reflect the corresponding KPIs
mycrosstab.columns = [
    "incidence rate",                    # Estimated incidence rate (new HIV infections per 1000 uninfected population)
    "annual deaths",                     # Estimated number of annual AIDS-related deaths
    "annual new infection",              # Estimated number of annual new HIV infections
    "people with HIV",                   # Estimated number of people living with HIV
    "annual death rate"                  # Estimated rate of annual AIDS-related deaths (per 100,000 population)
]

# Use Seaborn's heatmap to visualize the correlation between different KPIs in the dataset
# The heatmap will show the correlation matrix using Pearson's method, with annotations for the correlation values
import seaborn as se
se.heatmap(mycrosstab.corr(method="pearson"), annot=True)

# Define color codes for terminal output formatting to highlight text in different colors
class color:
    PURPLE = '\033[95m'
    CYAN = '\033[96m'
    DARKCYAN = '\033[36m'
    BLUE = '\033[94m'
    GREEN = '\033[92m'
    YELLOW = '\033[93m'
    RED = '\033[91m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'
    END = '\033[0m'

# Print a description of the KPIs being used, with formatted color output
print(color.BOLD + "incidence rate:" + color.END + color.RED + "Estimated incidence rate (new HIV infection per 1000 uninfected population)\n" + color.END,
      color.BOLD + "annual deaths:" + color.END + color.RED + "Estimated number of annual AIDS-related deaths\n" + color.END,
      color.BOLD + "annual new infection:" + color.END + color.RED + "Estimated number of annual new HIV infections\n" + color.END,
      color.BOLD + "people with HIV:" + color.END + color.RED + "Estimated number of people with HIV\n" + color.END,
      color.BOLD + "annual death rate:" + color.END + color.RED + "Estimated rate of annual AIDS-related deaths (per 100,000 population)" + color.END)

In [ ]:
#-------------------------------------------------------------------------------------------------------

In [ ]:
# Check the relationship between Key Performance Indicators (KPIs) in each UNICEF Region
# Filter the dataset to focus on global data (UNICEF Region = "Global"), for both sexes ("Both") 
# and for the age groups 0-14 and 15-19 years
Epid_DF_A3_Glob = Epid_DF[
    (Epid_DF["UNICEF Region"] == "Global") &                  # Focus on the global region
    (Epid_DF["Sex"] == "Both") &                              # Include both sexes (aggregate male and female)
    ((Epid_DF["Age"] == "Age 0-14") | (Epid_DF["Age"] == "Age 15-19"))  # Focus on age groups 0-14 and 15-19 years
]

# Create a cross-tabulation (mycrosstab) to compare the sum of 'Value' (number of people affected) 
# for each year, broken down by indicator (e.g., incidence rate, annual deaths, etc.)
mycrosstab = pd.crosstab(
    Epid_DF_A3_Glob["Year"],              # Group by 'Year'
    Epid_DF_A3_Glob["Indicator"],        # Group by 'Indicator' (e.g., annual deaths, annual new infections, etc.)
    values=Epid_DF_A3_Glob["Value"],     # Use the 'Value' column for aggregation
    aggfunc='sum'                         # Aggregate by summing the values for each combination of Year and Indicator
)

# Rename the columns for clarity to reflect the corresponding KPIs
mycrosstab.columns = [
    "incidence rate",                    # Estimated incidence rate (new HIV infections per 1000 uninfected population)
    "annual deaths",                     # Estimated number of annual AIDS-related deaths
    "annual new infection",              # Estimated number of annual new HIV infections
    "people with HIV",                   # Estimated number of people living with HIV
    "annual death rate"                  # Estimated rate of annual AIDS-related deaths (per 100,000 population)
]

# Use Seaborn's heatmap to visualize the correlation between different KPIs in the dataset
# The heatmap will show the correlation matrix using Pearson's method, with annotations for the correlation values
import seaborn as se
se.heatmap(mycrosstab.corr(method="pearson"), annot=True)

# Define color codes for terminal output formatting to highlight text in different colors
class color:
    PURPLE = '\033[95m'
    CYAN = '\033[96m'
    DARKCYAN = '\033[36m'
    BLUE = '\033[94m'
    GREEN = '\033[92m'
    YELLOW = '\033[93m'
    RED = '\033[91m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'
    END = '\033[0m'

# Print a description of the KPIs being used, with formatted color output
print(color.BOLD + "incidence rate:" + color.END + color.RED + "Estimated incidence rate (new HIV infection per 1000 uninfected population)\n" + color.END,
      color.BOLD + "annual deaths:" + color.END + color.RED + "Estimated number of annual AIDS-related deaths\n" + color.END,
      color.BOLD + "annual new infection:" + color.END + color.RED + "Estimated number of annual new HIV infections\n" + color.END,
      color.BOLD + "people with HIV:" + color.END + color.RED + "Estimated number of people with HIV\n" + color.END,
      color.BOLD + "annual death rate:" + color.END + color.RED + "Estimated rate of annual AIDS-related deaths (per 100,000 population)" + color.END)

In [ ]:
# Import plotly.express for interactive data visualization
import plotly.express as px

# Filter the dataset (preAdo_DF) to focus on:
# - "West and Central Africa" as the UNICEF Region
# - The indicator "Per cent with comprehensive, correct knowledge of HIV"
# - Data for the years 2020 and 2021 only
# - The disaggregation category as "Age"
# - The disaggregation (age groups) as either "15-19" or "20-24"
preAdo_DF_A2 = preAdo_DF[
    (preAdo_DF["UNICEF Region"] == "West and Central Africa") &   # Filter by region: West and Central Africa
    (preAdo_DF["Indicator"] == "Per cent with comprehensive, correct knowledge of HIV") &  # Filter by indicator: knowledge of HIV
    ((preAdo_DF["Year"] == 2021) | (preAdo_DF["Year"] == 2020)) &  # Filter for the years 2020 and 2021
    (preAdo_DF["DISAGG_CATEGORY"] == "Age") &  # Focus on age-based disaggregation
    ((preAdo_DF["DISAGG"] == "15-19") | (preAdo_DF["DISAGG"] == "20-24"))  # Focus on age groups 15-19 and 20-24
]

# Prepare the filtered DataFrame (preAdo_DF_A2) for visualization
df = preAdo_DF_A2

# Create an interactive sunburst chart using Plotly Express to show the distribution of knowledge percentage by year, sex, and age
# The sunburst chart will show the hierarchy of the data: Year -> Sex -> Disaggregation (Age group)
# The 'Value' column is used for the sizes of the segments
# Color is determined by the 'Value' column, and the color scale used is 'RdBu' (Red-Blue)
# The color midpoint is set to the weighted average of the 'Value' column for better visual balance
fig = px.sunburst(
    df,                                          # Data to visualize
    path=['Year', 'Sex', 'DISAGG'],              # Hierarchical structure: Year > Sex > Age group
    values='Value',                              # Size of segments based on the 'Value' column (percentage knowledge)
    color='Value',                               # Color segments based on the 'Value' column
    hover_data=['Value'],                        # Display 'Value' when hovering over segments
    color_continuous_scale='RdBu',                # Color scale (Red-Blue) for continuous data
    color_continuous_midpoint=np.average(df['Value'], weights=df['Value']),  # Color midpoint set to the weighted average of 'Value'
    title='Age of Percent with comprehensive, correct knowledge of HIV'  # Title of the chart
)

# Display the sunburst chart
fig.show()

In [ ]:
# Import plotly.express for creating interactive data visualizations
import plotly.express as px

# Filter the dataset (preAdo_DF) to focus on:
# - "West and Central Africa" as the UNICEF Region
# - The indicator "Per cent with comprehensive, correct knowledge of HIV"
# - Data for the years 2020 and 2021 only
# - The disaggregation category as "Education"
preAdo_DF_A1 = preAdo_DF[
    (preAdo_DF["UNICEF Region"] == "West and Central Africa") &   # Filter by region: West and Central Africa
    (preAdo_DF["Indicator"] == "Per cent with comprehensive, correct knowledge of HIV") &  # Filter by indicator: knowledge of HIV
    ((preAdo_DF["Year"] == 2021) | (preAdo_DF["Year"] == 2020)) &  # Filter for the years 2020 and 2021
    (preAdo_DF["DISAGG_CATEGORY"] == "Education")                  # Focus on education-based disaggregation
]

# Prepare the filtered DataFrame (preAdo_DF_A1) for visualization
df = preAdo_DF_A1

# Create an interactive sunburst chart using Plotly Express to show the distribution of knowledge percentage by year, sex, and education level
# The sunburst chart will show the hierarchy of the data: Year -> Sex -> Education level
# The 'Value' column is used for the sizes of the segments
# Color is determined by the 'Value' column, and the color scale used is 'RdBu' (Red-Blue)
# The color midpoint is set to the weighted average of the 'Value' column for better visual balance
fig = px.sunburst(
    df,                                          # Data to visualize
    path=['Year', 'Sex', 'DISAGG'],              # Hierarchical structure: Year > Sex > Education level
    values='Value',                              # Size of segments based on the 'Value' column (percentage knowledge)
    color='Value',                               # Color segments based on the 'Value' column
    hover_data=['Value'],                        # Display 'Value' when hovering over segments
    color_continuous_scale='RdBu',                # Color scale (Red-Blue) for continuous data
    color_continuous_midpoint=np.average(df['Value'], weights=df['Value']),  # Color midpoint set to the weighted average of 'Value'
    title="Education of Percent with comprehensive, correct knowledge of HIV"  # Title of the chart
)

# Display the sunburst chart
fig.show()

In [ ]:
# Import plotly.express for creating interactive data visualizations
import plotly.express as px

# Filter the dataset (preAdo_DF) to focus on:
# - "West and Central Africa" as the UNICEF Region
# - The indicator "Per cent with comprehensive, correct knowledge of HIV"
# - Data for the years 2020 and 2021 only
# - The disaggregation category as "Marital status"
preAdo_DF_A1 = preAdo_DF[
    (preAdo_DF["UNICEF Region"] == "West and Central Africa") &   # Filter by region: West and Central Africa
    (preAdo_DF["Indicator"] == "Per cent with comprehensive, correct knowledge of HIV") &  # Filter by indicator: knowledge of HIV
    ((preAdo_DF["Year"] == 2021) | (preAdo_DF["Year"] == 2020)) &  # Filter for the years 2020 and 2021
    (preAdo_DF["DISAGG_CATEGORY"] == "Marital status")              # Focus on marital status disaggregation
]

# Prepare the filtered DataFrame (preAdo_DF_A1) for visualization
df = preAdo_DF_A1

# Create an interactive sunburst chart using Plotly Express to show the distribution of knowledge percentage by year, sex, and marital status
# The sunburst chart will show the hierarchy of the data: Year -> Sex -> Marital status
# The 'Value' column is used for the sizes of the segments
# Color is determined by the 'Value' column, and the color scale used is 'RdBu' (Red-Blue)
# The color midpoint is set to the weighted average of the 'Value' column for better visual balance
fig = px.sunburst(
    df,                                          # Data to visualize
    path=['Year', 'Sex', 'DISAGG'],              # Hierarchical structure: Year > Sex > Marital status
    values='Value',                              # Size of segments based on the 'Value' column (percentage knowledge)
    color='Value',                               # Color segments based on the 'Value' column
    hover_data=['Value'],                        # Display 'Value' when hovering over segments
    color_continuous_scale='RdBu',                # Color scale (Red-Blue) for continuous data
    color_continuous_midpoint=np.average(df['Value'], weights=df['Value']),  # Color midpoint set to the weighted average of 'Value'
    title="Marital status of Percent with comprehensive, correct knowledge of HIV"  # Title of the chart
)

# Display the sunburst chart
fig.show()

In [ ]:
# Import plotly.express for creating interactive data visualizations
import plotly.express as px

# Filter the dataset (preAdo_DF) to focus on:
# - "West and Central Africa" as the UNICEF Region
# - The indicator "Per cent with comprehensive, correct knowledge of HIV"
# - Data for the years 2020 and 2021 only
# - The disaggregation category as "Residence"
preAdo_DF_A1 = preAdo_DF[
    (preAdo_DF["UNICEF Region"] == "West and Central Africa") &   # Filter by region: West and Central Africa
    (preAdo_DF["Indicator"] == "Per cent with comprehensive, correct knowledge of HIV") &  # Filter by indicator: knowledge of HIV
    ((preAdo_DF["Year"] == 2021) | (preAdo_DF["Year"] == 2020)) &  # Filter for the years 2020 and 2021
    (preAdo_DF["DISAGG_CATEGORY"] == "Residence")                  # Focus on residence disaggregation
]

# Prepare the filtered DataFrame (preAdo_DF_A1) for visualization
df = preAdo_DF_A1

# Create an interactive sunburst chart using Plotly Express to show the distribution of knowledge percentage by year, sex, and residence
# The sunburst chart will show the hierarchy of the data: Year -> Sex -> Residence
# The 'Value' column is used for the sizes of the segments
# Color is determined by the 'Value' column, and the color scale used is 'RdBu' (Red-Blue)
# The color midpoint is set to the weighted average of the 'Value' column for better visual balance
fig = px.sunburst(
    df,                                          # Data to visualize
    path=['Year', 'Sex', 'DISAGG'],              # Hierarchical structure: Year > Sex > Residence
    values='Value',                              # Size of segments based on the 'Value' column (percentage knowledge)
    color='Value',                               # Color segments based on the 'Value' column
    hover_data=['Value'],                        # Display 'Value' when hovering over segments
    color_continuous_scale='RdBu',                # Color scale (Red-Blue) for continuous data
    color_continuous_midpoint=np.average(df['Value'], weights=df['Value']),  # Color midpoint set to the weighted average of 'Value'
    title="Residence of Percent with comprehensive, correct knowledge of HIV"  # Title of the chart
)

# Display the sunburst chart
fig.show()

In [ ]:
# Import plotly.express for creating interactive data visualizations
import plotly.express as px

# Filter the dataset (preAdo_DF) to focus on:
# - "West and Central Africa" as the UNICEF Region
# - The indicator "Per cent with comprehensive, correct knowledge of HIV"
# - Data for the years 2020 and 2021 only
# - The disaggregation category as "Wealth quintile"
preAdo_DF_A1 = preAdo_DF[
    (preAdo_DF["UNICEF Region"] == "West and Central Africa") &  # Filter by region: West and Central Africa
    (preAdo_DF["Indicator"] == "Per cent with comprehensive, correct knowledge of HIV") &  # Filter by indicator: knowledge of HIV
    ((preAdo_DF["Year"] == 2021) | (preAdo_DF["Year"] == 2020)) &  # Filter for the years 2020 and 2021
    (preAdo_DF["DISAGG_CATEGORY"] == "Wealth quintile")  # Focus on wealth quintile disaggregation
]

# Prepare the filtered DataFrame (preAdo_DF_A1) for visualization
df = preAdo_DF_A1

# Create an interactive sunburst chart using Plotly Express to show the distribution of knowledge percentage by year, sex, and wealth quintile
# The sunburst chart will show the hierarchy of the data: Year -> Sex -> Wealth quintile
# The 'Value' column is used for the sizes of the segments
# Color is determined by the 'Value' column, and the color scale used is 'RdBu' (Red-Blue)
# The color midpoint is set to the weighted average of the 'Value' column for better visual balance
fig = px.sunburst(
    df,                                          # Data to visualize
    path=['Year', 'Sex', 'DISAGG'],              # Hierarchical structure: Year > Sex > Wealth quintile
    values='Value',                              # Size of segments based on the 'Value' column (percentage knowledge)
    color='Value',                               # Color segments based on the 'Value' column
    hover_data=['Value'],                        # Display 'Value' when hovering over segments
    color_continuous_scale='RdBu',                # Color scale (Red-Blue) for continuous data
    color_continuous_midpoint=np.average(df['Value'], weights=df['Value']),  # Color midpoint set to the weighted average of 'Value'
    title="Wealth quintile of Percent with comprehensive, correct knowledge of HIV"  # Title of the chart
)

# Display the sunburst chart
fig.show()

In [ ]:
#-------------------------------------------------------------------------------

In [ ]:
# Filter the dataset (preAdo_DF) for:
# - "West and Central Africa" as the UNICEF Region
# - The indicator "Per cent with comprehensive, correct knowledge of HIV"
# - Exclude data where the sex is "Both"
# - The disaggregation category as "Age"
# - Focus on the age group "15-19"
DF1 = preAdo_DF[
    (preAdo_DF["UNICEF Region"] == "West and Central Africa") &  # Filter by region: West and Central Africa
    (preAdo_DF["Indicator"] == "Per cent with comprehensive, correct knowledge of HIV") &  # Filter by indicator: HIV knowledge
    (preAdo_DF["Sex"] != "Both") &  # Exclude "Both" sex category
    (preAdo_DF["DISAGG_CATEGORY"] == "Age") &  # Filter by age disaggregation
    (preAdo_DF["DISAGG"] == "15-19")  # Focus on age group 15-19
]

# Filter the dataset (Epid_DF) for:
# - "West and Central Africa" as the UNICEF Region
# - The indicator "Estimated number of annual AIDS-related deaths"
# - Exclude data where the sex is "Both"
# - Focus on the age group "15-19"
DF2 = Epid_DF[
    (Epid_DF["UNICEF Region"] == "West and Central Africa") &  # Filter by region: West and Central Africa
    (Epid_DF["Indicator"] == "Estimated number of annual AIDS-related deaths") &  # Filter by indicator: annual AIDS-related deaths
    (Epid_DF["Sex"] != "Both") &  # Exclude "Both" sex category
    (Epid_DF["Age"] == "Age 15-19")  # Focus on age group 15-19
]

# Merge the two datasets (DF1 and DF2) on common columns 'Year' and 'Sex'
# This combines the knowledge of HIV (Education) and the number of annual AIDS-related deaths (Death) data for each year and sex
Merge = pd.merge(DF1, DF2, on=['Year', 'Sex'])

# Keep only the relevant columns from the merged dataset:
# - 'Value_x' (Education) from DF1
# - 'Value_y' (Death) from DF2
Merge = Merge[['Value_x', 'Value_y']]

# Rename the columns for better clarity:
# - 'Value_x' becomes "Education"
# - 'Value_y' becomes "Death"
Merge.columns = ["Education", "Death"]

# Calculate the Pearson correlation coefficient between the 'Education' (HIV knowledge) and 'Death' (AIDS-related deaths)
Cor = Merge.corr(method='pearson')

# Print the correlation result to observe the relationship
print(Cor)

# The correlation result shows the relationship between the level of education (HIV knowledge) and the number of annual AIDS-related deaths.
# A positive correlation indicates that higher education (correct knowledge of HIV) is associated with lower deaths, and vice versa.
# This suggests that better education on HIV could have a direct effect on reducing AIDS-related deaths.